# Clustering Algorithm Selection

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.utils import resample
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score, davies_bouldin_score

df_unscaled = pd.read_csv("../data/interim/weather_per_country.csv", index_col=0)
df_standard_scaled = pd.read_csv("../data/processed/X_standard_scaled.csv", index_col=0)
df_tsne = pd.read_csv("../data/processed/X_tsne.csv", index_col=0)
df_umap = pd.read_csv("../data/processed/X_umap.csv", index_col=0)

data = {
    "unscaled": df_unscaled,
    "standard_scaled": df_standard_scaled,
    "tsne": df_tsne,
    "umap": df_umap,
}

## Systematic Algorithm Selection Framework

Select a clustering algorithm based on the sample size and the number of features in the dataset.

In [2]:
def systematic_algorithm_selection(data):
    """Simple framework for algorithm selection"""
    n_samples, n_features = data.shape

    print("ALGORITHM SELECTION FRAMEWORK")
    print("=" * 30)

    # Step 1: Data size assessment
    if n_samples < 500:
        size_category = "Small - all algorithms feasible"
    elif n_samples < 5000:
        size_category = "Medium - prefer K-means or DBSCAN"
    else:
        size_category = "Large - K-means strongly preferred"

    print(f"Data size: {n_samples} samples ({size_category})")

    # Step 2: Dimensionality assessment
    if n_features < 10:
        dim_category = "Low dimensional - no issues"
    else:
        dim_category = "High dimensional - consider preprocessing"

    print(f"Features: {n_features} ({dim_category})")

    # Step 3: Simple recommendation
    if n_samples > 10000:
        recommendation = "K-means (scalable)"
    elif n_samples < 1000:
        recommendation = "Hierarchical (rich analysis)"
    else:
        recommendation = "K-means or DBSCAN (depends on cluster shapes)"

    print(f"Recommendation: {recommendation}")

    return recommendation


# Example usage
for key, df in data.items():
    print(f"\nDataset: {key}")
    print(systematic_algorithm_selection(df))


Dataset: unscaled
ALGORITHM SELECTION FRAMEWORK
Data size: 185 samples (Small - all algorithms feasible)
Features: 12 (High dimensional - consider preprocessing)
Recommendation: Hierarchical (rich analysis)
Hierarchical (rich analysis)

Dataset: standard_scaled
ALGORITHM SELECTION FRAMEWORK
Data size: 185 samples (Small - all algorithms feasible)
Features: 10 (High dimensional - consider preprocessing)
Recommendation: Hierarchical (rich analysis)
Hierarchical (rich analysis)

Dataset: tsne
ALGORITHM SELECTION FRAMEWORK
Data size: 185 samples (Small - all algorithms feasible)
Features: 1 (Low dimensional - no issues)
Recommendation: Hierarchical (rich analysis)
Hierarchical (rich analysis)

Dataset: umap
ALGORITHM SELECTION FRAMEWORK
Data size: 185 samples (Small - all algorithms feasible)
Features: 1 (Low dimensional - no issues)
Recommendation: Hierarchical (rich analysis)
Hierarchical (rich analysis)


## Data Characteristics Analysis

In [3]:
def analyze_data_characteristics(data):
    """Analyze key data characteristics for algorithm selection"""

    if isinstance(data, pd.DataFrame):
        df = data
    else:
        df = pd.DataFrame(data)

    print("DATA CHARACTERISTICS ANALYSIS")
    print("=" * 30)

    # Basic stats
    n_samples, n_features = df.shape
    print(f"Samples: {n_samples:,}")
    print(f"Features: {n_features}")

    # Missing data
    missing_pct = (df.isnull().sum().sum() / df.size) * 100
    print(f"Missing data: {missing_pct:.1f}%")

    # Data types
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    print(f"Numeric features: {len(numeric_cols)}")
    print(f"Categorical features: {len(categorical_cols)}")

    # Scale differences (for numeric data
    if len(numeric_cols) > 1:
        ranges = df[numeric_cols].max() - df[numeric_cols].min()
        max_range = ranges.max()
        min_range = ranges[ranges > 0].min()
        scale_ratio = max_range / min_range if min_range > 0 else 0
        print(f"Scale difference ratio: {scale_ratio:.1f}")

        if scale_ratio > 100:
            print("⚠️  Large scale differences - scaling recommended")

    # High-level recommendations
    issues = []
    if missing_pct > 20:
        issues.append("High missing data")
    if len(categorical_cols) > len(numeric_cols):
        issues.append("Mostly categorical data")
    if n_features > 20:
        issues.append("High dimensionality")

    if issues:
        print("Issues to address:", ", ".join(issues))
    else:
        print("✅ Data looks good for clustering")

    return {
        "n_samples": n_samples,
        "n_features": n_features,
        "missing_pct": missing_pct,
        "n_numeric": len(numeric_cols),
        "n_categorical": len(categorical_cols),
    }


for key, df in data.items():
    print(f"\nDataset: {key}")
    analyze_data_characteristics(df)


Dataset: unscaled
DATA CHARACTERISTICS ANALYSIS
Samples: 185
Features: 12
Missing data: 0.0%
Numeric features: 12
Categorical features: 0
Scale difference ratio: 2950.0
⚠️  Large scale differences - scaling recommended
✅ Data looks good for clustering

Dataset: standard_scaled
DATA CHARACTERISTICS ANALYSIS
Samples: 185
Features: 10
Missing data: 0.0%
Numeric features: 10
Categorical features: 0
Scale difference ratio: 1.6
✅ Data looks good for clustering

Dataset: tsne
DATA CHARACTERISTICS ANALYSIS
Samples: 185
Features: 1
Missing data: 0.0%
Numeric features: 1
Categorical features: 0
✅ Data looks good for clustering

Dataset: umap
DATA CHARACTERISTICS ANALYSIS
Samples: 185
Features: 1
Missing data: 0.0%
Numeric features: 1
Categorical features: 0
✅ Data looks good for clustering


## Computational Resource Assessment: Matching Algorithms to Available Resources

We don't need to take this into account for our dataset since they are small.

## Cross-Validation for Algorithm Selection

### Why It Matters
Clustering lacks ground truth labels, so traditional cross-validation doesn’t apply directly. Instead, assess stability and reproducibility across data splits and algorithm parameter settings to choose algorithms that produce consistent patterns.

### Approaches:

1. **Bootstrap Stability**: Resample data with replacement, cluster each sample, measure agreement (Adjusted Rand Index) between runs
2. **Subsample Validation**: Split data into train/validation, fit on train, apply to validation (e.g., PCA+K-means)
3. **Parameter Robustness**: Vary key parameters (K for K-means, eps for DBSCAN), check whether cluster assignments change drastically

We are going to implement the bootstrap stability approach.

In [4]:
def bootstrap_stability(data, cluster_func, n_runs=10):
    """Assess clustering stability via bootstrap"""
    labels_list = []
    for i in range(n_runs):
        # Resample indices to keep track of mapping to original data
        sample = resample(data, random_state=i)
        sample_indices = sample.index if hasattr(sample, "index") else None
        labels = cluster_func(sample)
        # Store both labels and indices for later alignment
        labels_list.append((labels, sample_indices))
    # Compute pairwise ARI
    aris = []
    for i in range(n_runs):
        for j in range(i + 1, n_runs):
            # Align labels by sorting according to sample indices
            labels_i, idx_i = labels_list[i]
            labels_j, idx_j = labels_list[j]
            if idx_i is not None and idx_j is not None:
                # Find intersection of indices and align
                common_idx = np.intersect1d(idx_i, idx_j)
                idx_i_aligned = [np.where(idx_i == ix)[0][0] for ix in common_idx]
                idx_j_aligned = [np.where(idx_j == ix)[0][0] for ix in common_idx]
                aligned_labels_i = labels_i[idx_i_aligned]
                aligned_labels_j = labels_j[idx_j_aligned]
                aris.append(adjusted_rand_score(aligned_labels_i, aligned_labels_j))
            else:
                aris.append(adjusted_rand_score(labels_i, labels_j))
    return np.mean(aris)


# Usage with K-means
def km_labels(data):
    return KMeans(n_clusters=4, random_state=42).fit_predict(data)


# Usage with DBSCAN
def dbscan_labels(data):
    return DBSCAN(eps=0.5).fit_predict(data)


# Usage with Hierarchical (Agglomerative)
def hierarchical_labels(data):
    return AgglomerativeClustering(n_clusters=4).fit_predict(data)


k_means_stability = bootstrap_stability(data["umap"], km_labels, n_runs=20)
print(f"K-means stability (ARI): {k_means_stability:.3f}")

dbscan_stability = bootstrap_stability(data["umap"], dbscan_labels, n_runs=20)
print(f"DBSCAN stability (ARI): {dbscan_stability:.3f}")

hierarchical_stability = bootstrap_stability(
    data["umap"], hierarchical_labels, n_runs=20
)
print(f"Hierarchical stability (ARI): {hierarchical_stability:.3f}")

K-means stability (ARI): 0.687
DBSCAN stability (ARI): 0.900
Hierarchical stability (ARI): 0.742


## Cross validation insights

<!-- K-means stability (ARI): 0.687
DBSCAN stability (ARI): 0.900
Hierarchical stability (ARI): 0.742 -->

This is a interesting result and after investigating I found, that DBSCAN only seems stable because in most of the cases it clusters into one big cluster. This is not a useful clustering result which leads me to the conclusion that K-means and Hierarchical clustering make sense for our dataset.

In [5]:
def evaluate_algorithms(data, algorithms):
    """Evaluate multiple clustering algorithms on key metrics"""
    results = {}
    for name, labels in algorithms.items():
        if len(set(labels)) > 1:
            sil = silhouette_score(data, labels)
            db = davies_bouldin_score(data, labels)
        else:
            sil, db = -1, np.inf
        results[name] = {"silhouette": sil, "db_index": db}
        print(f"{name}: Silhouette={sil:.3f}, DB-Index={db:.3f}")
    return results


# Example usage
alg_results = {
    "K-means": KMeans(n_clusters=4).fit_predict(data["umap"]),
    "DBSCAN": DBSCAN(eps=0.5).fit_predict(data["umap"]),
    "Hierarchical": AgglomerativeClustering(n_clusters=4).fit_predict(data["umap"]),
}
metrics = evaluate_algorithms(data["umap"], alg_results)
best = min(metrics, key=lambda x: metrics[x]["db_index"])
print(f"Best algorithm by DB-Index: {best}")

K-means: Silhouette=0.587, DB-Index=0.492
DBSCAN: Silhouette=-1.000, DB-Index=inf
Hierarchical: Silhouette=0.572, DB-Index=0.484
Best algorithm by DB-Index: Hierarchical


In [ ]:
def run_basic_clustering_suite(scaled_data):
    """Run basic version of each algorithm for initial assessment"""

    print("\nRUNNING BASIC CLUSTERING SUITE")
    print("-" * 30)

    results = {}

    # K-means with K=4 (common starting point)
    kmeans = KMeans(n_clusters=4, random_state=42)
    km_labels = kmeans.fit_predict(scaled_data)
    km_score = silhouette_score(scaled_data, km_labels)
    results["kmeans"] = {"labels": km_labels, "silhouette": km_score, "n_clusters": 4}
    print(f"✅ K-means (K=4): Silhouette = {km_score:.3f}")

    # DBSCAN with reasonable defaults
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    db_labels = dbscan.fit_predict(scaled_data)
    db_n_clusters = len(set(db_labels)) - (1 if -1 in db_labels else 0)
    if db_n_clusters > 1:
        db_score = silhouette_score(scaled_data, db_labels)
    else:
        db_score = -1
    results["dbscan"] = {
        "labels": db_labels,
        "silhouette": db_score,
        "n_clusters": db_n_clusters,
    }
    print(f"✅ DBSCAN: {db_n_clusters} clusters, Silhouette = {db_score:.3f}")

    # Hierarchical with K=4
    hier = AgglomerativeClustering(n_clusters=4)
    hier_labels = hier.fit_predict(scaled_data)
    hier_score = silhouette_score(scaled_data, hier_labels)
    results["hierarchical"] = {
        "labels": hier_labels,
        "silhouette": hier_score,
        "n_clusters": 4,
    }
    print(f"✅ Hierarchical (K=4): Silhouette = {hier_score:.3f}")

    return results


# Example usage
basic_results = run_basic_clustering_suite(data["umap"])


RUNNING BASIC CLUSTERING SUITE
------------------------------
✅ K-means (K=4): Silhouette = 0.597


TypeError: DBSCAN.__init__() got an unexpected keyword argument 'n_clusters'